In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import RawBoschCNCDataloader

### Create dataloaders
- First create raw dataloader and load raw h5 files, save metadata as pkl file
- Second create an stft dataloader from this

In [3]:
# metadata path ist der pfad wo der dataloader das metadata file abspeichern wird
base_loader = RawBoschCNCDataloader()
base_loader.load_raw_h5_data(raw_h5_data_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/raw_data/data")
base_loader.save_metadata_to_file(metadata_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/metadata_ws4096.pkl")
base_loader.generate_datasets_by_size(train_size=.5,val_size=.1,test_size=.4)
stft_base_loader = base_loader.get_windowed_samples_as_stft_dataloader()

Loading dataset files:   0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/42978 [00:00<?, ?it/s]

### Normalize data for CNN Training

In [4]:
channel_means = np.mean(stft_base_loader.X_train, axis=(0, 2, 3), keepdims=True)
channel_stds = np.std(stft_base_loader.X_train, axis=(0, 2, 3), keepdims=True)

X_train_scaled = (stft_base_loader.X_train - channel_means) / channel_stds
X_val_scaled = (stft_base_loader.X_val - channel_means) / channel_stds
X_test_scaled = (stft_base_loader.X_test - channel_means) / channel_stds

In [6]:
import torch
from torch import tensor
from torchmetrics.classification import (
    BinaryAUROC,
    BinaryF1Score,
    BinaryAveragePrecision)
from drift_ml.datasets.bosch_cnc_machining.models.nnclassifier import NNClassifier
from drift_ml.datasets.bosch_cnc_machining.models.lenet import LeNet

torch.manual_seed(1)
np.random.seed(0)

auroc = BinaryAUROC()
auprc = BinaryAveragePrecision()
f1 = BinaryF1Score(threshold=0.5)

model = NNClassifier(model=LeNet, device="cuda")
model.fit(
    X_train_scaled, 
    stft_base_loader.y_train[:, np.newaxis], 
    X_val_scaled, 
    stft_base_loader.y_val[:, np.newaxis], 
    lrate=1e-2,
    epochs=50,
    verbose=True
)

DEBUG:root:Starting training with batch size 64, lrate 0.01, epochs 50


Training NN:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 